In [1]:
# packages gerais
import pandas as pd
import joblib

# df_sales_ dataset
df_sales_filtered_all = joblib.load('df_sales_filtered_all.pkl')
df_sales_filtered_2018_2019 = joblib.load('df_sales_filtered_2018_2019.pkl')
df_sales_filtered_2019 = joblib.load('df_sales_filtered_2019.pkl')
df_sales_filtered_last_6_month = joblib.load('df_sales_filtered_last_6_month.pkl')

In [2]:
df_sales_filtered_all

,store_id,product_id,year,week,sales,revenue,stock,price
0,S0020,P0001,2017,45,0.0,0.00,10.0,7.75
1,S0020,P0001,2017,46,1.0,6.57,9.0,7.75
2,S0020,P0001,2017,47,2.0,13.14,7.0,7.75
3,S0020,P0001,2017,48,0.0,0.00,7.0,6.75
4,S0020,P0001,2017,49,1.0,5.72,6.0,6.75
...,...,...,...,...,...,...,...,...
198309,S0097,P0748,2019,36,2.0,28.00,10.0,18.90
198310,S0097,P0748,2019,37,0.0,0.00,10.0,18.90
198311,S0097,P0748,2019,38,1.0,17.50,9.0,18.90
198312,S0097,P0748,2019,39,1.0,17.50,8.0,18.90


In [3]:
df_sales_filtered_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198314 entries, 0 to 198313
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   store_id    198314 non-null  object 
 1   product_id  198314 non-null  object 
 2   year        198314 non-null  UInt32 
 3   week        198314 non-null  UInt32 
 4   sales       198314 non-null  float64
 5   revenue     198314 non-null  float64
 6   stock       198314 non-null  float64
 7   price       198314 non-null  float64
dtypes: UInt32(2), float64(4), object(2)
memory usage: 11.0+ MB


In [4]:
# Agrupar por product_id e somar as previsões de vendas
df_sales_filtered_all_sum = df_sales_filtered_all.groupby('product_id')['sales'].sum()

# Ordenar os produtos pela soma das previsões de vendas em ordem decrescente
top_products_all = df_sales_filtered_all_sum.sort_values(ascending=False).head(10)

# Exibir os top 10 produtos que irão mais vender
print(top_products_all)

product_id
P0103    196230.000
P0438    120806.000
P0131     47440.000
P0364     34139.000
P0500     31383.000
P0051     30793.000
P0413     27037.626
P0388     26372.000
P0333     25780.000
P0348     22384.000
Name: sales, dtype: float64


In [6]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Filtrar os dados para incluir apenas semanas anteriores à semana 41
df_sales_filtered_all = df_sales_filtered_all[df_sales_filtered_all['week'] < 41]

# Lista para armazenar os resultados das previsões
forecast_results = []

# Identificar os product_ids únicos
product_ids = df_sales_filtered_all['product_id'].unique()

# Realizar previsões para cada product_id
for product_id in product_ids:
    product_sales = df_sales_filtered_all[df_sales_filtered_all['product_id'] == product_id]
    
    # Converter 'year' e 'week' para datetime
    product_sales['date'] = pd.to_datetime(product_sales['year'].astype(str) + '-W' + product_sales['week'].astype(str) + '-1', format='%Y-W%W-%w')
    
    # Definir 'date' como índice
    product_sales.set_index('date', inplace=True)
    
    # Verificar se há dados suficientes para ajustar o modelo ARIMA
    if len(product_sales) < 2:
        continue  # Pular este product_id se não houver dados suficientes

    # Fit ARIMA model (ajuste os parâmetros p, d, q conforme necessário)
    try:
        model = ARIMA(product_sales['sales'], order=(5, 1, 0))
        model_fit = model.fit()
        
        # Prever as vendas para as próximas 4 semanas
        forecast_steps = 4
        forecast = model_fit.forecast(steps=forecast_steps)
        
        # Criar um DataFrame para armazenar as previsões
        forecast_dates = pd.date_range(start=product_sales.index[-1] + pd.Timedelta(days=7), periods=forecast_steps, freq='W-MON')
        forecast_df = pd.DataFrame({'product_id': product_id, 'date': forecast_dates, 'sales_forecast': forecast})
        
        # Adicionar as previsões à lista de resultados
        forecast_results.append(forecast_df)
    
    except Exception as e:
        print(f"Não foi possível ajustar o modelo ARIMA para o product_id {product_id}: {e}")

# Concatenar todos os resultados em um único DataFrame
forecast_results_df = pd.concat(forecast_results)



# Exibir as previsões
print(forecast_results_df)


C:\Users\fabia\AppData\Local\Temp\ipykernel_5204\1869579924.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_sales['date'] = pd.to_datetime(product_sales['year'].astype(str) + '-W' + product_sales['week'].astype(str) + '-1', format='%Y-W%W-%w')
c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. f

Não foi possível ajustar o modelo ARIMA para o product_id P0305: too many indices for array: array is 0-dimensional, but 1 were indexed


C:\Users\fabia\AppData\Local\Temp\ipykernel_5204\1869579924.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_sales['date'] = pd.to_datetime(product_sales['year'].astype(str) + '-W' + product_sales['week'].astype(str) + '-1', format='%Y-W%W-%w')
c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. f

Não foi possível ajustar o modelo ARIMA para o product_id P0630: too many indices for array: array is 0-dimensional, but 1 were indexed


c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\fabia\AppData\Local\Temp\ipykernel_5204\1869579924.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_sales['date'] = pd.to_datetime(product_sales['year'].astype(str) + '-W

Não foi possível ajustar o modelo ARIMA para o product_id P0710: too many indices for array: array is 0-dimensional, but 1 were indexed
                    product_id       date  sales_forecast
246                      P0001 2019-09-02        0.257608
247                      P0001 2019-09-09        0.252707
248                      P0001 2019-09-16        0.153909
249                      P0001 2019-09-23        0.230229
491                      P0005 2018-03-26        0.521918
...                        ...        ...             ...
2019-11-04 00:00:00      P0725 2019-11-04        0.194680
2019-10-14 00:00:00      P0155 2019-10-14        1.214263
2019-10-21 00:00:00      P0155 2019-10-21        0.397016
2019-10-28 00:00:00      P0155 2019-10-28       -0.079918
2019-11-04 00:00:00      P0155 2019-11-04        0.136106

[2208 rows x 3 columns]


c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
# Arredondar os valores da coluna 'sales_forecast' para o inteiro mais próximo
forecast_results_df['sales_forecast'] = forecast_results_df['sales_forecast'].round()

# Mostrar o DataFrame com os valores arredondados
print(forecast_results_df)


                    product_id       date  sales_forecast
246                      P0001 2019-09-02             0.0
247                      P0001 2019-09-09             0.0
248                      P0001 2019-09-16             0.0
249                      P0001 2019-09-23             0.0
491                      P0005 2018-03-26             1.0
...                        ...        ...             ...
2019-11-04 00:00:00      P0725 2019-11-04             0.0
2019-10-14 00:00:00      P0155 2019-10-14             1.0
2019-10-21 00:00:00      P0155 2019-10-21             0.0
2019-10-28 00:00:00      P0155 2019-10-28            -0.0
2019-11-04 00:00:00      P0155 2019-11-04             0.0

[2208 rows x 3 columns]


In [ ]:
# Filtrar o DataFrame para mostrar apenas os sales_forecast maiores que 10
filtered_forecast = forecast_results_df[forecast_results_df['sales_forecast'] > 10]

# Mostrar as primeiras 60 linhas do DataFrame filtrado
print(filtered_forecast.head(60))

    product_id       date  sales_forecast
158      P0026 2019-10-14            14.0
159      P0026 2019-10-21            17.0
160      P0026 2019-10-28            21.0
161      P0026 2019-11-04            17.0
595      P0055 2019-10-28            11.0
98       P0058 2018-10-01            15.0
99       P0058 2018-10-08            15.0
100      P0058 2018-10-15            14.0
101      P0058 2018-10-22            21.0
599      P0103 2019-10-14            68.0
600      P0103 2019-10-21            79.0
601      P0103 2019-10-28            58.0
602      P0103 2019-11-04            69.0
600      P0129 2019-10-14            13.0
601      P0129 2019-10-21            13.0
602      P0129 2019-10-28            12.0
603      P0129 2019-11-04            11.0
477      P0130 2019-10-14            26.0
478      P0130 2019-10-21            45.0
479      P0130 2019-10-28            41.0
480      P0130 2019-11-04            49.0
600      P0131 2019-10-14            23.0
601      P0131 2019-10-21         

In [ ]:
# Agrupar por product_id e somar as previsões de vendas
product_forecast_sum = forecast_results_df.groupby('product_id')['sales_forecast'].sum()

# Ordenar os produtos pela soma das previsões de vendas em ordem decrescente
top_products = product_forecast_sum.sort_values(ascending=False).head(10)

# Exibir os top 10 produtos que irão mais vender
print(top_products)


product_id
P0397    6418.0
P0559     896.0
P0438     673.0
P0103     274.0
P0130     161.0
P0363     126.0
P0388     122.0
P0500     121.0
P0131     115.0
P0182     113.0
Name: sales_forecast, dtype: float64


In [ ]:
# Comparar os dois rankings
comparison_df = pd.DataFrame({
    'Top Produtos Históricos': top_products_all.index,
    'Vendas Históricas': top_products_all.values,
    'Top Produtos Previsões': top_products.index,
    'Previsões de Vendas': top_products.values
})

print("\nComparação entre top 10 produtos historicamente mais vendidos e com maiores previsões de vendas:")
print(comparison_df)


Comparação entre top 10 produtos historicamente mais vendidos e com maiores previsões de vendas:
  Top Produtos Históricos  Vendas Históricas Top Produtos Previsões  \
0                   P0103         196230.000                  P0397   
1                   P0438         120806.000                  P0559   
2                   P0131          47440.000                  P0438   
3                   P0364          34139.000                  P0103   
4                   P0500          31383.000                  P0130   
5                   P0051          30793.000                  P0363   
6                   P0413          27037.626                  P0388   
7                   P0388          26372.000                  P0500   
8                   P0333          25780.000                  P0131   
9                   P0348          22384.000                  P0182   

   Previsões de Vendas  
0               6418.0  
1                896.0  
2                673.0  
3                274